In [0]:
service_credential = dbutils.secrets.get(scope="akv_db_scope_training", key="dbtrainingspnsecret")
# Connecting to ADLS Gen 2
spark.conf.set("fs.azure.account.auth.type.azstoragetraining.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.azstoragetraining.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.azstoragetraining.dfs.core.windows.net", "3f91e76c-33f1-4ec5-bb48-a65ceebe2b98")
spark.conf.set("fs.azure.account.oauth2.client.secret.azstoragetraining.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.azstoragetraining.dfs.core.windows.net", "https://login.microsoftonline.com/45da25de-8003-4abf-bea8-55aba615e5e7/oauth2/token")

In [0]:
#list out all files in container inside-air-bnb-dataset
dbutils.fs.ls("abfss://inside-airbnb-dataset@azstoragetraining.dfs.core.windows.net")

In [0]:
#file paths
listings_file_path = "abfss://inside-airbnb-dataset@azstoragetraining.dfs.core.windows.net/albany-listings.csv"
reviews_file_path = "abfss://inside-airbnb-dataset@azstoragetraining.dfs.core.windows.net/albany-reviews.csv"

#read all files into dataframes(csv format files)
listings = (spark.read.format("csv") 
    .option("header", "true") 
    .option("inferSchema", "true") 
    .load(listings_file_path) )
# display(listings)

reviews = (spark.read.format("csv") 
    .option("header", "true") 
    .option("inferSchema", "true") 
    .load(reviews_file_path) )
# display(reviews)

In [0]:
%sql
SHOW CATALOGS

In [0]:
%sql
USE CATALOG az_adb_simbus_training;
CREATE SCHEMA IF NOT EXISTS az_adb_simbus_training.inside_air_bnb_project_schema;

In [0]:
#write to delta format and saving tables to schema inside_air_bnb_project_schema

listings.write.format("delta").mode("overwrite").saveAsTable("inside_air_bnb_project_schema.bronze_listings");
reviews.write.format("delta").mode("overwrite").saveAsTable("inside_air_bnb_project_schema.bronze_reviews");

In [0]:
tables = spark.sql("SHOW TABLES IN inside_air_bnb_project_schema")
display(tables)

In [0]:
bronze_listings = spark.table("inside_air_bnb_project_schema.bronze_listings")
display(bronze_listings)